In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 get_column_descriptions	 
load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 save_dataframes	 store_objects	 
sys	 



# More webscraping

In [22]:

wiki_url = 'https://ageofempires.fandom.com/wiki/'
tables_url = '{}Villager_(Age_of_Empires_II)'.format(wiki_url)
tables_df_list = pd.read_html(tables_url)
print(len(tables_df_list))
villager_gathering_rate_df = tables_df_list[1]

4


In [21]:

villager_gathering_rate_df

,Villager type,Resource,Gathering rate (resources/second)
0,Farmer,Food,0.53*
1,Fisherman,Food,0.43
2,Hunter,Food,0.41
3,Lumberjack,Wood,0.39
4,Gold Miner,Gold,0.38
5,Stone Miner,Stone,0.36
6,Shepherd,Food,0.33
7,Forager,Food,0.31


In [ ]:

tables_url = '{}Civilizations_(Age_of_Empires_II)'.format(wiki_url)
tables_df_list = pd.read_html(tables_url)
len(tables_df_list)

In [ ]:

tables_df_list[0]

In [ ]:

tables_df_list[2]

In [ ]:

file_path = os.path.join(html_folder, 'military_unit_links.html')
with open(file_path, 'rb') as f:
    html_doc = f.read().decode('utf-8')
soup = BeautifulSoup(html_doc, 'html.parser')


# Scraping the Availability Stats

In [52]:

summary_box_url = 'https://ageofempires.fandom.com/wiki/Age_of_Empires_II:_The_Age_of_Kings'
with urllib.request.urlopen(summary_box_url) as response:
    summary_box_html = response.read()
summary_box_soup = BeautifulSoup(summary_box_html, 'html.parser')
summary_box_selector = '#mw-content-text > table[class="navbox"]:nth-child(114)'
summary_box_table = summary_box_soup.select(summary_box_selector)[0]

In [226]:

tr_soup_dict = {}
tr_list = []
th_list = ['The Age of Kings', 'Infantry', 'Archers', 'Cavalry', 'Siege', 'Navy', 'Unique',
           'The Conquerors', 'The Forgotten', 'The African Kingdoms', 'Rise of the Rajas']
military_units_list = ['Infantry', 'Archers', 'Cavalry', 'Siege', 'Navy', 'Unique']
heading_list = []
for summary_box_th in summary_box_table.find_all('th'):
    for heading in th_list:
        if heading in summary_box_th.text:
            tr_soup_dict[heading] = summary_box_th.parent
            if heading in military_units_list:
                tr_list.append(tr_soup_dict[heading])
            else:
                tr_list.append(tr_soup_dict[heading].next_sibling)
            heading_list.append(heading)

In [216]:

def get_link_html(link):
    link_url = 'https://ageofempires.fandom.com{}'.format(link['href'])
    if '_(Age_of_Empires_II)' in link_url:
        with urllib.request.urlopen(link_url) as response:
            link_html = response.read()
    else:
        try:
            extended_link_url = '{}_(Age_of_Empires_II)'.format(link_url)
            with urllib.request.urlopen(extended_link_url) as response:
                link_html = response.read()
            link_url = extended_link_url
        except:
            with urllib.request.urlopen(link_url) as response:
                link_html = response.read()
    
    return link_url, link_html

In [265]:

def process_all_except(appl_civ_text, row_dict):
    for civ_name in civ_name_list:
        if civ_name in appl_civ_text:
            row_dict[civ_name] = 0
        else:
            row_dict[civ_name] = 1
    
    return row_dict

In [266]:

def process_article(link_soup, row_dict):
    art_table = art_table_list[0]
    art_ul_list = art_table.find_all('ul')
    assert len(art_ul_list) == 2

    # Get available civs
    available_ul = art_ul_list[0]
    for link in available_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 1

    # Get unavailable civs
    unavailable_ul = art_ul_list[1]
    for link in unavailable_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 0

    return row_dict

In [267]:

space_regex = re.compile(r'\s+')
def process_small_list_of_units(appl_civ_text, row_dict):
    appl_civ_list = space_regex.split(appl_civ_text)
    for civ_name in civ_name_list:
        row_dict[civ_name] = 0
    for civ_name in appl_civ_list:
        if civ_name in civ_name_list:
            row_dict[civ_name] = 1
    
    return row_dict

In [268]:

toc_selector = '#toc'
def process_exceptions(link_soup, row_dict):
    desc_p_list = link_soup.select(toc_selector)
    if len(desc_p_list):
        for sib in desc_p_list[0].previous_siblings:
            if 'available to all civilizations except for' in str(sib):
                break
        process_all_except(sib.text.strip(), row_dict)
    
    return row_dict

In [273]:

article_selector = '#mw-content-text > table.article-table'
def process_unit(link_soup, appl_civ_text, row_dict):
    for column_name in civ_name_list:
        columns_set.add(column_name)
    if 'All civilizations' in appl_civ_text:
        for civ_name in civ_name_list:
            row_dict[civ_name] = 1
    elif 'All except ' in appl_civ_text:
        row_dict = process_all_except(appl_civ_text, row_dict)
    elif 'See ' in appl_civ_text:
        art_table_list = link_soup.select(article_selector)
        if len(art_table_list):
            row_dict = process_article(link_soup, row_dict)
        else:
            row_dict = process_exceptions(link_soup, row_dict)
    else:
        row_dict = process_small_list_of_units(appl_civ_text, row_dict)
    
    return row_dict

In [290]:

def consume_aside(unit_soup, row_dict):
    aside_list = unit_soup.select('#mw-content-text > aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list)
                    if column_name == 'Civilization':
                        row_dict = process_unit(unit_soup, column_value, row_dict)
                    else:
                        columns_set.add(column_name)
                        row_dict[column_name] = column_value
    
    return row_dict

In [291]:

appl_civ_selector = '#mw-content-text > aside > section > div[data-source="Civilization"] > div'
title_selector = '#PageHeader > div.page-header__main > h1'
def get_unit_pages(soup, heading):
    for link in soup.find_all('a'):
        unit_row_dict = {'Heading': heading}
        unit_url, unit_html = get_link_html(link)
        unit_soup = BeautifulSoup(unit_html, 'html.parser')
        unit_title = unit_soup.select(title_selector)[0].string.split('(')[0].strip()
        index_list.append(unit_title)
        rows_list.append(consume_aside(unit_soup, unit_row_dict))

In [292]:

rows_list = []
index_list = []
columns_set = set(['Heading'])
for soup, heading in zip(tr_list, heading_list):
    get_unit_pages(soup, heading)

In [293]:

other_columns_set = columns_set - set(civ_name_list)
other_columns_set = other_columns_set - set(['Heading'])
columns_list = ['Heading'] + list(other_columns_set) + civ_name_list
availability_df = pd.DataFrame(data=rows_list, index=index_list, columns=columns_list)
print(availability_df.shape)
availability_df[availability_df.isnull().any(axis=1)].shape

(104, 61)


(104, 61)

In [295]:

availability_df.loc['Monk', other_columns_set].T

Required                                                     NaN
PSpeed                                                       NaN
AttackBonus                                                  NaN
FD                                                           NaN
Wood                                                         NaN
UpTime                                                       NaN
UpTo                                                         NaN
Type                                                 Monk Healer
MinRange                                                     NaN
Range          Conversion : 9 vs units 0 vs buildings , Trebu...
PierceArmor                                                    0
Speed                                                        0.7
TrainTime                                             51 seconds
UpCost                                                       NaN
Age                                                   Castle Age
Garrison                 

In [ ]:

def f(x):
    x = str(x)
    if ':' in x or ',' in x:
        
        return True
    
    else:
        
        return False

In [302]:

structured_list = []
for column_name in availability_df.columns:
    if availability_df[column_name].map(f).any():
        structured_list.append(column_name)
availability_df[structured_list].sample(5).T

,Capped Ram,Huskarl,Trebuchet,War Elephant,Bombard Cannon
AttackBonus,+150 vs Building +50 vs Siege weapon,"+6 , +10 (Elite) vs Archer +2, +3 (Elite) vs ...",+250 vs Building (unpacked),"+7, +10 (Elite) vs Building +7, +10 (Elite) vs...",+200 vs Building +40 vs Ship +40 vs Stone defe...
FD,NaN,NaN,6 (unpacked),NaN,7
Range,NaN,NaN,16,NaN,12
PierceArmor,190,"6 , 8 (Elite)",8 (packed) 150 (unpacked),"2, 3 (Elite)",5
TrainTime,36 seconds,16 seconds,50 seconds,31 seconds,56 seconds
UpCost,"1,000 food","1,200 food , 550 gold",NaN,"1,600 food , 1,200 gold",NaN
HP,200,"60, 70 (Elite)",150,"450, 600 (Elite)",80
LOS,3,"3, 5 (Elite)",19,"4, 5 (Elite)",14
Attack,3 melee,"10, 12 (Elite) melee",200 pierce (unpacked),"15, 20 (Elite) melee",40 melee
ROF,5,2.03,10 (unpacked),2.03,6.5


In [303]:

file_path = os.path.join(DATA_CSV_FOLDER, 'availability_df.csv')
availability_df.to_csv(file_path)

In [311]:

save_dataframes(availability_df=availability_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\availability_df.csv


In [312]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'availability_df.xlsx')
availability_df.to_excel(file_path)

In [310]:

# All military units are created 15% faster
unit_type_list = availability_df['Type'].unique().tolist()
civilian_list = [unit_type for unit_type in unit_type_list if 'Civilian' in str(unit_type)]
match_series = ~availability_df['Type'].isin(civilian_list) & (availability_df['Aztecs'] == 1)
availability_df.loc[availability_df[match_series].index, 'TrainTime']

Monk                     51 seconds
Transport Ship           46 seconds
Militia                  21 seconds
Man-at-Arms              21 seconds
Long Swordsman           21 seconds
Two-Handed Swordsman     21 seconds
Champion                 21 seconds
Spearman                 22 seconds
Pikeman                  22 seconds
Archer                   35 seconds
Crossbowman              27 seconds
Arbalest                 27 seconds
Skirmisher               22 seconds
Elite Skirmisher         22 seconds
Hand Cannoneer           34 seconds
Heavy Cavalry Archer     27 seconds
Paladin                  30 seconds
Camel                    22 seconds
Heavy Camel              22 seconds
Battering Ram            36 seconds
Capped Ram               36 seconds
Siege Ram                36 seconds
Mangonel                 46 seconds
Onager                   46 seconds
Siege Onager             46 seconds
Scorpion                 30 seconds
Heavy Scorpion           30 seconds
Bombard Cannon           56 

In [316]:

training_time_list = availability_df['TrainTime'].unique().tolist()
complex_list = [x for x in training_time_list if ',' in str(x)]
match_series = (availability_df['TrainTime'].isin(complex_list))
index_list = availability_df[match_series].index.tolist()
index_list = index_list + ['Elite {}'.format(idx) for idx in index_list]
availability_df.loc[index_list, 'TrainTime']

C:\Users\dev\Anaconda3\lib\site-packages\pandas\core\indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


Chu Ko Nu                    16 , 13 (Elite) seconds
Genoese Crossbowman           22, 19 (Elite) seconds
Boyar                         23, 20 (Elite) seconds
Elite Chu Ko Nu                                  NaN
Elite Genoese Crossbowman                        NaN
Elite Boyar                                      NaN
Name: TrainTime, dtype: object

In [317]:

[fn for fn in dir(availability_df) if 'map' in fn or 'apply' in fn]

['apply', 'applymap']

In [326]:

elite_df = availability_df.applymap(lambda x: 'Elite' in str(x))
elite_set = set(['Elite {}'.format(idx) for idx in elite_df[elite_df.any(axis=1)].index])
elite_set - set(availability_df.index)

{'Elite Teutonic Knight', 'Elite Chu Ko Nu', 'Elite Boyar', 'Elite Rattan Archer', 'Elite Longboat', 'Elite Woad Raider', 'Elite War Wagon', 'Elite Ballista Elephant', 'Elite Berserk', 'Elite Janissary', 'Elite Samurai', 'Elite Shotel Warrior', 'Elite Jaguar Warrior', 'Elite Tarkan', 'Elite Plumed Archer', 'Elite Genoese Crossbowman', 'Elite Turtle Ship', 'Elite Camel Archer', 'Elite Karambit Warrior', 'Elite Magyar Huszar', 'Elite Imperial Skirmisher', 'Elite Elephant Archer', 'Elite Cataphract', 'Elite Kamayuk', 'Elite Caravel', 'Elite Gbeto', 'Elite Huskarl', 'Elite Longbowman', 'Elite Organ Gun', 'Elite Conquistador', 'Elite Arambai', 'Elite War Elephant', 'Elite Genitour', 'Elite Mameluke', 'Elite Throwing Axeman', 'Elite Mangudai'}

In [345]:

super_availability_df = availability_df.copy()
for column_name in (elite_set - set(availability_df.index)):
    match_series = (availability_df.index == column_name[6:])
    df = availability_df[match_series].copy()
    df['UpTo'] = ''
    df.index = [column_name]
    super_availability_df = pd.concat([super_availability_df, df])
save_dataframes(include_index=True, super_availability_df=super_availability_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv


In [423]:

dataframes_dict = load_dataframes(super_availability_df=super_availability_df)
super_availability_df = dataframes_dict['super_availability_df']
super_availability_df = super_availability_df.set_index('Unnamed: 0')
super_availability_df.index.name = 'Unit Name'

Attempting to load C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\pickle\super_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv.


In [425]:

print(super_availability_df.index)
super_availability_df.columns

Index(['Arambai', 'Arbalest', 'Archer', 'Archer units', 'Ballista Elephant',
       'Battering Ram', 'Battle Elephant', 'Berserk', 'Bombard Cannon',
       'Boyar',
       ...
       'Transport Ship', 'Trebuchet', 'Turtle Ship', 'Two-Handed Swordsman',
       'Unique unit', 'Villager', 'War Elephant', 'War Galley', 'War Wagon',
       'Woad Raider'],
      dtype='object', name='Unit Name', length=140)


Index(['Heading', 'Required', 'PSpeed', 'AttackBonus', 'FD', 'Wood', 'UpTime',
       'UpTo', 'Type', 'MinRange', 'Range', 'PierceArmor', 'Speed',
       'TrainTime', 'UpCost', 'Age', 'Garrison', 'HP', 'Intro', 'LOS', 'Food',
       'UpFrom', 'AOE', 'Attack', 'ROF', 'Class', 'Building', 'Gold',
       'Accuracy', 'Armor', 'Britons', 'Byzantines', 'Celts', 'Chinese',
       'Franks', 'Goths', 'Japanese', 'Mongols', 'Persians', 'Saracens',
       'Teutons', 'Turks', 'Vikings', 'Aztecs', 'Huns', 'Koreans', 'Mayans',
       'Spanish', 'Incas', 'Indians', 'Italians', 'Magyars', 'Slavs',
       'Berbers', 'Ethiopians', 'Malians', 'Portuguese', 'Burmese', 'Khmer',
       'Malay', 'Vietnamese'],
      dtype='object')

In [426]:

elite_df = super_availability_df.applymap(lambda x: 'Elite' in str(x))
elite_df.columns[elite_df.any(axis=0)]

Index(['AttackBonus', 'UpTo', 'UpFrom', 'Attack', 'ROF', 'Class', 'Accuracy',
       'Armor'],
      dtype='object')

In [427]:

def f(row_series):
    attack_bonus = str(row_series['AttackBonus'])
    if 'Elite' in row_series.name:
        attack_bonus = elite_regex.sub(r' \g<2>\g<3> ', attack_bonus)
    else:
        attack_bonus = elite_regex.sub(r' \g<2>\g<1> ', attack_bonus)
    
    return attack_bonus

elite_regex = re.compile(r' *\+?(\d+) *, +(\+?)(\d+) +\(Elite\) *')
match_series = super_availability_df['AttackBonus'].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df['AttackBonus'] = super_availability_df.apply(f, axis=1)

# ['+20 vs War elephant +8, +12 (Elite) vs Cavalry +6 , +10  (Elite) vs Camel']
super_availability_df.loc[idx_series, 'AttackBonus'].tolist()

['+20 vs War elephant +12 vs Cavalry +10 vs Camel']

In [428]:

def f(row_series):
    attack = str(row_series['Attack'])
    if 'Elite' in row_series.name:
        attack = elite_regex.sub(r'\g<2>\g<3> ', attack)
    else:
        attack = elite_regex.sub(r'\g<2>\g<1> ', attack)
    
    return attack

elite_regex = re.compile(r' *\+?(\d+) *, +(\+?)(\d+) +\(Elite\) *')
match_series = super_availability_df['Attack'].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df['Attack'] = super_availability_df.apply(f, axis=1)

# ['8 , 10 (Elite)  melee', '6, 8 (Elite)  pierce', '16, 20 (Elite)  pierce',
#  '8 , 11 (Elite)  melee', '7, 8 (Elite)  melee', '15, 20 (Elite)  melee']
super_availability_df.loc[idx_series, 'Attack'].tolist()

['10 melee', '8 pierce', '20 pierce', '11 melee', '8 melee', '20 melee']

In [429]:

column_name = 'ROF'
def f(row_series):
    column_value = str(row_series[column_name])
    if 'Elite' in row_series.name:
        column_value = elite_regex.sub(r' \g<2>\g<3> ', column_value)
    else:
        column_value = elite_regex.sub(r' \g<2>\g<1> ', column_value)
    
    return column_value.strip()

elite_regex = re.compile(r' *\+?([.\d]+) *, +(\+?)([.\d]+) +\(Elite\) *')
match_series = super_availability_df[column_name].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df[column_name] = super_availability_df.apply(f, axis=1)

# ['1.83 , 1.73  (Elite)', '3.65, 3.84 (Elite)', '1.83 , 1.73  (Elite)', '3.65, 3.84 (Elite)',
#  '3.05, 2.03 (Elite)', '3.05, 2.03 (Elite)']
super_availability_df.loc[idx_series, column_name].tolist()

['1.83', '3.65', '1.73', '3.84', '2.03', '3.05']

In [430]:

column_name = 'Class'
def f(row_series):
    column_value = str(row_series[column_name])
    if 'Elite' in row_series.name:
        column_value = elite_regex.sub(r'(\g<3>)', column_value)
    else:
        column_value = only_elite_regex.sub(r' \g<1> ', column_value)
        column_value = elite_regex.sub(r'(\g<2>)', column_value)
    
    return column_value.strip()

elite_regex = re.compile(r'\(((\+[.\d]+), +)?(\+[.\d]+) +\(Elite\) +\)')
only_elite_regex = re.compile(r' +([^)(]+) \((\+[.\d]+) +\(Elite\) +\) +')
match_series = super_availability_df[column_name].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df[column_name] = super_availability_df.apply(f, axis=1)

# ['Cavalry (+12, +16 (Elite) ) Unique unit', 'Cavalry (+12, +16 (Elite) ) Unique unit',
#  'Ship (+8, +11 (Elite) ) Turtle Ship (+1 (Elite) ) Gunpowder unit Unique unit',
#  'Ship (+8, +11 (Elite) ) Turtle Ship (+1 (Elite) ) Gunpowder unit Unique unit']
super_availability_df.loc[idx_series, column_name].tolist()

['Cavalry (+12) Unique unit', 'Cavalry (+16) Unique unit', 'Ship (+11) Turtle Ship (+1) Gunpowder unit Unique unit', 'Ship (+8) Turtle Ship Gunpowder unit Unique unit']

In [432]:

column_name = 'Accuracy'
def f(row_series):
    column_value = str(row_series[column_name])
    if 'Elite' in row_series.name:
        column_value = elite_regex.sub(r'\g<2>%', column_value)
    else:
        column_value = elite_regex.sub(r'\g<1>%', column_value)
    
    return column_value.strip()

elite_regex = re.compile(r'(\d+)%, (\d+)% \(Elite\)')
match_series = super_availability_df[column_name].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df[column_name] = super_availability_df.apply(f, axis=1)

# ['20%, 30% (Elite)', '65%, 70% (Elite)', '20%, 30% (Elite)', '65%, 70% (Elite)',
#  '70%, 80% (Elite)', '80%, 90% (Elite)', '80%, 90% (Elite)', '70%, 80% (Elite)',
#  '80%, 90% (Elite)', '80%, 90% (Elite)']
super_availability_df.loc[idx_series, column_name].tolist()

['20%', '65%', '30%', '70%', '80%', '90%', '90%', '70%', '80%', '80%']

In [434]:

column_name = 'Armor'
def f(row_series):
    column_value = str(row_series[column_name])
    if 'Elite' in row_series.name:
        column_value = elite_regex.sub(r'\g<2>%', column_value)
    else:
        column_value = elite_regex.sub(r'\g<1>%', column_value)
    
    return column_value.strip()

elite_regex = re.compile(r'(\d+), (\d+) \(Elite\)')
match_series = super_availability_df[column_name].map(lambda x: 'Elite' in str(x))
idx_series = super_availability_df[match_series].index
super_availability_df[column_name] = super_availability_df.apply(f, axis=1)

# ['0, 2 (Elite)', '4, 6 (Elite)', '0, 1 (Elite)', '0, 2 (Elite)', '4, 6 (Elite)',
#  '0, 1 (Elite)', '1, 2 (Elite)', '1, 2 (Elite)', '0, 1 (Elite)', '0, 1 (Elite)',
#  '0, 1 (Elite)', '0, 1 (Elite)', '5, 10 (Elite)', '0, 1 (Elite)', '6, 8 (Elite)',
#  '1, 2 (Elite)', '1, 2 (Elite)', '0, 1 (Elite)', '0, 1 (Elite)', '0, 1 (Elite)',
#  '0, 1 (Elite)', '5, 10 (Elite)', '0, 1 (Elite)', '6, 8 (Elite)']
super_availability_df.loc[idx_series, column_name].tolist()

['0%', '4%', '0%', '2%', '6%', '1%', '2%', '2%', '1%', '1%', '1%', '1%', '10%', '1%', '8%', '1%', '1%', '0%', '0%', '0%', '0%', '5%', '0%', '6%']

In [436]:

save_dataframes(include_index=True, super_availability_df=super_availability_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv


In [452]:

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = super_availability_df.applymap(get_mixed)
digit_df.columns[digit_df.any(axis=0)]

In [456]:

regex_list = [r'Archer', r'Building', r'Building +\(unpacked\)', r'Camel', r'Castle', r'Cavalry',
              r'Cavalry +archer', r'Condottiero', r'Eagle +Warrior', r'Gunpowder +unit',
              r'Infantry', r'Monk', r'Ram', r'Ship', r'Siege +weapon', r'Spearman',
              r'Standard +building', r'Stone +defense', r'Turtle +Ship', r'Unique +unit',
              r'Wall +and +gate', r'War +elephant']
column_name = 'AttackBonus'
regex_regex = re.compile(r'[ +(\\]+')
for versus_regex in regex_list:
    new_column_name = regex_regex.sub('_', versus_regex)
    new_column_name = re.sub(r'\)', '', new_column_name)
    new_column_name = '{}vs{}'.format(column_name, new_column_name)
    versus_regex = re.compile(r'\+(\d+) +vs +{}'.format(versus_regex))
    def f(x):
        match_obj = versus_regex.search(x)
        if match_obj:
            attack_bonus_versus = int(match_obj.group(1))
        else:
            attack_bonus_versus = 0

        return attack_bonus_versus

    super_availability_df[new_column_name] = super_availability_df[column_name].map(f)

In [457]:

save_dataframes(include_index=True, super_availability_df=super_availability_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv


In [464]:

dataframes_dict = load_dataframes(super_availability_df=super_availability_df)
super_availability_df = dataframes_dict['super_availability_df']
super_availability_df = super_availability_df.set_index('Unit Name')
super_availability_df.index.name = 'Unit Name'

Attempting to load C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\pickle\super_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv.


In [465]:

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = super_availability_df.applymap(get_mixed)
digit_df.columns[digit_df.any(axis=0)]

Index(['AttackBonus', 'UpTime', 'Range', 'Speed', 'TrainTime', 'UpCost', 'LOS',
       'Attack', 'Class', 'Gold', 'Accuracy'],
      dtype='object')

In [467]:

column_name = 'UpTime'
def f(x):
    x = str(x)
    match_obj = re.search(r'\d+', x)
    if match_obj:
        up_time = int(match_obj.group())
    else:
        up_time = 0
    
    return up_time

super_availability_df[column_name] = super_availability_df[column_name].map(f)

In [492]:

save_dataframes(include_index=True, super_availability_df=super_availability_df)
file_path = os.path.join(xlsx_folder, 'super_availability_df.xlsx')
super_availability_df.to_excel(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\csv\super_availability_df.csv


In [489]:

column_name = 'Range'
new_column_name = 'attack_or_conversion_vs_units'
def f(x):
    
    return float(re.search(r'[.\d]+', str(x)).group())

super_availability_df[new_column_name] = super_availability_df[column_name].map(f)

In [491]:

x = 'Conversion : 7 vs units 0 vs buildings , Trebuchets , Battering Rams Healing: 4'
match_list = re.findall(r'[.\d]+', str(x))
match_list[min()]

['7', '0', '4']

In [482]:

super_availability_df[column_name].unique()

array(['5', '4', 0, '12', '13', '6', '15', '7', '8', '1', '3', '2.49',
       'Conversion : 7 vs units 0 vs buildings , Trebuchets , Battering Rams Healing: 4',
       'Conversion : 9 vs units 0 vs buildings , Trebuchets , Battering Rams Healing: 4',
       '0', '16'], dtype=object)

In [481]:

super_availability_df[column_name] = super_availability_df[column_name].fillna(value=0)
match_series = super_availability_df[column_name].isnull()
super_availability_df[match_series].index

Index([], dtype='object', name='Unit Name')